In [ ]:
import re

def clean_latex_string(s):
    # Remove leading and trailing $
    s = s.strip('$')
    # Remove all backslashes
    s = s.replace('\\', '')
    # Replace 'pm' (from '\pm' or any residue) with ±
    s = re.sub(r'\s*pm\s*', ' +- ', s)
    return s

def latex_to_results(latex_str):
    tabular_match = re.search(r'\\begin{tabular}{.*?}(.*?)\\end{tabular}', latex_str, re.DOTALL)
    if not tabular_match:
        raise ValueError("No tabular block found.")
    tabular_str = tabular_match.group(1)

    results = []
    for line in tabular_str.splitlines():
        line = line.strip()
        if (
            not line
            or line.startswith("\\")
            or '&' not in line
            or 'Pair' in line
            or 'Score' in line
        ):
            continue

        # Remove trailing \\
        if line.endswith('\\\\'):
            line = line[:-2].strip()
        elif line.endswith('\\'):
            line = line[:-1].strip()

        cols = [c.strip() for c in line.split('&')]
        if len(cols) != 7:
            continue  # Skip malformed lines

        # Parse columns
        pair = clean_latex_string(cols[0])
        cointegration_score_str = cols[1]

        sci_match = re.match(
            r'^\$?([-\d\.]+)\\times\s*10\^\{(-?\d+)\}\$?$', cointegration_score_str)
        if sci_match:
            cointegration_score = float(sci_match.group(1)) * (10 ** int(sci_match.group(2)))
        else:
            try:
                cointegration_score = float(cointegration_score_str.replace('$', '').replace('%', '').strip())
            except:
                cointegration_score = cointegration_score_str

        val_mse = float(cols[2])
        test_mse = float(cols[3])
        yoy_str = clean_latex_string(cols[4])
        gt_yoy = clean_latex_string(cols[5])
        returns_score = float(cols[6])

        results.append((pair, cointegration_score, val_mse, test_mse, yoy_str, gt_yoy, returns_score))
    return results


import re

def print_averages(results):
    results_clean = [r for r in results if r[-1] != -1 and r[-1] <= 1.0]  

    val_mse_list = []
    test_mse_list = []
    yoy_mean_list = []
    yoy_stddev_list = []
    gt_yoy_list = []
    returns_score_list = []

    for r in results_clean:
        _, _, val_mse, test_mse, yoy_str, gt_yoy_str, returns_score = r

        # Validation and test MSE
        val_mse_list.append(float(val_mse))
        test_mse_list.append(float(test_mse))

        # yoy_str, e.g., '3.11% +- 0.23%'
        yoy_match = re.match(r'([-0-9.]+)% \+\- ([0-9.]+)%', yoy_str)
        if yoy_match:
            yoy_mean = float(yoy_match.group(1))
            yoy_stddev = float(yoy_match.group(2))
            yoy_mean_list.append(yoy_mean)
            yoy_stddev_list.append(yoy_stddev)
        else:
            # Handle if not in expected format (fallback to nan)
            yoy_mean_list.append(float('nan'))
            yoy_stddev_list.append(float('nan'))

        # gt_yoy, e.g., '3.31%' or '-100.00%'
        try:
            gt_yoy = float(gt_yoy_str.replace('%', ''))
        except:
            gt_yoy = float('nan')
        gt_yoy_list.append(gt_yoy)

        # returns_score
        returns_score_list.append(float(returns_score))

    import numpy as np

    def safe_mean(lst):
        arr = np.array(lst)
        return f"{np.nanmean(arr):.5f}"
    
    def safe_std(lst):
        arr = np.array(lst)
        return f"{np.nanstd(arr):.5f}"

    def safe_median(lst):
        arr = np.array(lst)
        return f"{np.nanmedian(arr):.5f}"

    print("Average (std dev) val_mse:", safe_mean(val_mse_list), "+-", safe_std(val_mse_list))
    print("Average (std dev) test_mse:", safe_mean(test_mse_list), "+-", safe_std(test_mse_list))
    print("Average yoy mean:", safe_mean(yoy_mean_list))
    print("Average yoy stddev:", safe_mean(yoy_stddev_list))
    print("Average gt_yoy:", safe_mean(gt_yoy_list))
    print("Average (std dev) returns_score:", safe_mean(returns_score_list), "+-", safe_std(returns_score_list))
    
    print("Median val_mse:", safe_median(val_mse_list))
    print("Median test_mse:", safe_median(test_mse_list))
    print("Median yoy mean:", safe_median(yoy_mean_list))
    print("Median yoy stddev:", safe_median(yoy_stddev_list))
    print("Median gt_yoy:", safe_median(gt_yoy_list))
    print("Median returns_score:", safe_median(returns_score_list))

    
    

In [42]:
transformer_str = r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
1. (PFF,EMB) & $1.38\times 10^{-4}$ & 0.14459491 & 0.25138518 & $3.11\% \pm 0.23\%$ & 3.31\% & 1.00 \\
2. (IFGL,MBB) & $4.45\times 10^{-4}$ & 0.40354022 & 0.25973642 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
3. (IFGL,EMB) & $7.27\times 10^{-4}$ & 0.54631454 & 0.6438404 & $21.96\% \pm 0.22\%$ & 11.61\% & 1.09 \\
4. (IGIB,IEI) & $7.97\times 10^{-4}$ & 0.059546065 & 0.16654141 & $-8.56\% \pm 0.34\%$ & 1.38\% & 0.90 \\
5. (IGF,DVY) & $8.40\times 10^{-4}$ & 0.063635066 & 0.2158686 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
6. (USIG,IEI) & $1.09\times 10^{-3}$ & 0.4056698 & 0.96185297 & $-86.57\% \pm 5.78\%$ & -63.21\% & 0.36 \\
7. (IFGL,BND) & $1.20\times 10^{-3}$ & 1.623811 & 1.4397576 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
8. (IFGL,SMH) & $1.64\times 10^{-3}$ & 3.247992 & 37.484634 & $-100.00\% \pm 0.00\%$ & 1482.49\% & 0.00 \\
9. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 2.6418028 & 41.7105 & $-100.00\% \pm 0.00\%$ & 455.31\% & 0.00 \\
10. (IGF,PPH) & $2.29\times 10^{-3}$ & 0.16840412 & 1.5937397 & $43.77\% \pm 1.35\%$ & -100.00\% & -1 \\
11. (PFF,MBB) & $2.58\times 10^{-3}$ & 0.28943416 & 0.31500983 & $7.39\% \pm 0.16\%$ & 6.35\% & 1.01 \\
12. (IGF,PKW) & $2.68\times 10^{-3}$ & 0.092310764 & 7.2816553 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
13. (IFGL,PHO) & $2.70\times 10^{-3}$ & 3.1937869 & 31.942844 & $-100.00\% \pm 0.00\%$ & 262.45\% & 0.00 \\
14. (IFGL,QQQ) & $2.76\times 10^{-3}$ & 2.0910814 & 27.769453 & $-100.00\% \pm 0.00\%$ & 3566.11\% & 0.00 \\
15. (PFF,BND) & $2.78\times 10^{-3}$ & 0.32102343 & 0.14645025 & $1.32\% \pm 0.14\%$ & 2.88\% & 0.98 \\
16. (PFF,SHY) & $3.04\times 10^{-3}$ & 0.52075595 & 2.4110925 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
17. (IFGL,ONEQ) & $3.49\times 10^{-3}$ & 4.6872334 & 36.20738 & $-100.00\% \pm 0.00\%$ & 389.24\% & 0.00 \\
18. (IGF,FTCS) & $3.83\times 10^{-3}$ & 0.027053414 & 0.53847057 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
19. (SHY,BND) & $4.05\times 10^{-3}$ & 0.12681825 & 0.19713683 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
20. (IGF,PEY) & $4.10\times 10^{-3}$ & 0.061120767 & 0.08792867 & $-76.04\% \pm 5.23\%$ & -58.46\% & 0.58 \\
21. (IFGL,PKW) & $4.77\times 10^{-3}$ & 1.0091032 & 8.79016 & $307.51\% \pm 2.48\%$ & 1039.39\% & 0.36 \\
22. (IFGL,FTCS) & $5.00\times 10^{-3}$ & 1.068998 & 13.741698 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
23. (IGF,PDP) & $5.27\times 10^{-3}$ & 0.040064145 & 0.37759876 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
24. (IGSB,SHV) & $5.29\times 10^{-3}$ & 0.40211874 & 0.05034999 & $2.08\% \pm 0.23\%$ & 1.24\% & 1.01 \\
25. (IFGL,USIG) & $5.35\times 10^{-3}$ & 0.4676678 & 0.35168847 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
26. (IFGL,PDP) & $6.21\times 10^{-3}$ & 0.26517487 & 8.212122 & $-100.00\% \pm 0.00\%$ & 901.14\% & 0.00 \\
27. (IGF,BBH) & $6.33\times 10^{-3}$ & 0.09602024 & 0.038710304 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
28. (IFGL,SHY) & $6.79\times 10^{-3}$ & 0.6948471 & 0.95719296 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
29. (IFGL,PRFZ) & $7.87\times 10^{-3}$ & 1.8801084 & 10.015597 & $-100.00\% \pm 0.00\%$ & 161.27\% & 0.00 \\
30. (IEF,USIG) & $7.90\times 10^{-3}$ & 1.2662985 & 16.705374 & $0.00\% \pm 0.00\%$ & 21.30\% & 0.82 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

"""

kalman_str = r"""


\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
1. (PFF,EMB) & $1.38\times 10^{-4}$ & 4.99826 & 13.49622 & $-82.63\% \pm 30.20\%$ & 3.31\% & 0.17 \\
2. (IFGL,MBB) & $4.45\times 10^{-4}$ & 11.91925 & 9.53453 & $4.84\% \pm 0.34\%$ & -100.00\% & -1 \\
3. (IFGL,EMB) & $7.27\times 10^{-4}$ & 2.44704 & 5.87790 & $-43.49\% \pm 5.95\%$ & 11.61\% & 0.51 \\
4. (IGIB,IEI) & $7.97\times 10^{-4}$ & 7.04674 & 6.27775 & $7.87\% \pm 0.55\%$ & 1.38\% & 1.06 \\
5. (IGF,DVY) & $8.40\times 10^{-4}$ & 2.45625 & 11.10707 & $47.01\% \pm 6.71\%$ & -100.00\% & -1 \\
6. (USIG,IEI) & $1.09\times 10^{-3}$ & 8.24087 & 19.50070 & $7.47\% \pm 0.44\%$ & -63.21\% & 2.92 \\
7. (IFGL,BND) & $1.20\times 10^{-3}$ & 9.86063 & 11.34749 & $3.15\% \pm 0.25\%$ & -100.00\% & -1 \\
8. (IFGL,SMH) & $1.64\times 10^{-3}$ & 0.52055 & 1.17276 & $204.55\% \pm 14.00\%$ & 1482.49\% & 0.19 \\
9. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 0.71553 & 2.43627 & $125.59\% \pm 9.99\%$ & 455.31\% & 0.41 \\
10. (IGF,PPH) & $2.29\times 10^{-3}$ & 1.53290 & 1.78350 & $27.48\% \pm 2.39\%$ & -100.00\% & -1 \\
11. (PFF,MBB) & $2.58\times 10^{-3}$ & 1.59049 & 3.74586 & $7.78\% \pm 0.67\%$ & 6.35\% & 1.01 \\
12. (IGF,PKW) & $2.68\times 10^{-3}$ & 1.44136 & 4.66554 & $61.50\% \pm 3.09\%$ & -100.00\% & -1 \\
13. (IFGL,PHO) & $2.70\times 10^{-3}$ & 6.60638 & 5.60842 & $41.01\% \pm 2.49\%$ & 262.45\% & 0.39 \\
14. (IFGL,QQQ) & $2.76\times 10^{-3}$ & 0.73231 & 2.51582 & $457.32\% \pm 12.12\%$ & 3566.11\% & 0.15 \\
15. (PFF,BND) & $2.78\times 10^{-3}$ & 1.84901 & 2.63888 & $6.04\% \pm 0.79\%$ & 2.88\% & 1.03 \\
16. (PFF,SHY) & $3.04\times 10^{-3}$ & 4.00318 & 14.92111 & $0.76\% \pm 0.07\%$ & -100.00\% & -1 \\
17. (IFGL,ONEQ) & $3.49\times 10^{-3}$ & 1.01884 & 2.01991 & $53.62\% \pm 1.43\%$ & 389.24\% & 0.31 \\
18. (IGF,FTCS) & $3.83\times 10^{-3}$ & 2.66918 & 11.64964 & $31.31\% \pm 4.28\%$ & -100.00\% & -1 \\
19. (SHY,BND) & $4.05\times 10^{-3}$ & 4.56110 & 15.86956 & $3.81\% \pm 0.80\%$ & -100.00\% & -1 \\
20. (IGF,PEY) & $4.10\times 10^{-3}$ & 1.65948 & 2.54887 & $6.99\% \pm 0.88\%$ & -58.46\% & 2.58 \\
21. (IFGL,PKW) & $4.77\times 10^{-3}$ & 5.61737 & 3.14836 & $106.69\% \pm 4.89\%$ & 1039.39\% & 0.18 \\
22. (IFGL,FTCS) & $5.00\times 10^{-3}$ & 11.16817 & 7.40878 & $59.05\% \pm 3.47\%$ & -100.00\% & -1 \\
23. (IGF,PDP) & $5.27\times 10^{-3}$ & 1.05944 & 2.00898 & $67.33\% \pm 4.13\%$ & -100.00\% & -1 \\
24. (IGSB,SHV) & $5.29\times 10^{-3}$ & 41.36759 & 5.67130 & $-52.98\% \pm 8.53\%$ & 1.24\% & 0.46 \\
25. (IFGL,USIG) & $5.35\times 10^{-3}$ & 11.67894 & 15.51695 & $2.28\% \pm 0.20\%$ & -100.00\% & -1 \\
26. (IFGL,PDP) & $6.21\times 10^{-3}$ & 4.25741 & 2.05212 & $95.38\% \pm 3.32\%$ & 901.14\% & 0.20 \\
27. (IGF,BBH) & $6.33\times 10^{-3}$ & 3.47911 & 0.80322 & $10.05\% \pm 1.00\%$ & -100.00\% & -1 \\
28. (IFGL,SHY) & $6.79\times 10^{-3}$ & 7.54957 & 14.03887 & $3.65\% \pm 0.26\%$ & -100.00\% & -1 \\
29. (IFGL,PRFZ) & $7.87\times 10^{-3}$ & 6.18911 & 3.45879 & $18.85\% \pm 1.78\%$ & 161.27\% & 0.45 \\
30. (IEF,USIG) & $7.90\times 10^{-3}$ & 1.93007 & 3.41579 & $3.17\% \pm 0.34\%$ & 21.30\% & 0.85 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
"""

time_moe_str = r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
1. (PFF,EMB) & $1.38\times 10^{-4}$ & 0.10627 & 0.09116 & $2.18\% \pm 0.59\%$ & 3.31\% & 0.99 \\
2. (IFGL,MBB) & $4.45\times 10^{-4}$ & 0.19378 & 0.10622 & $-98.69\% \pm 31.54\%$ & -100.00\% & -1 \\
3. (IFGL,EMB) & $7.27\times 10^{-4}$ & 0.09751 & 0.08337 & $-24.99\% \pm 52.26\%$ & 11.61\% & 0.67 \\
4. (IGIB,IEI) & $7.97\times 10^{-4}$ & 0.07641 & 0.07921 & $1.72\% \pm 0.22\%$ & 1.38\% & 1.00 \\
5. (IGF,DVY) & $8.40\times 10^{-4}$ & 0.05064 & 0.13567 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
6. (USIG,IEI) & $1.09\times 10^{-3}$ & 0.09513 & 0.18667 & $-83.67\% \pm 29.83\%$ & -63.21\% & 0.44 \\
7. (IFGL,BND) & $1.20\times 10^{-3}$ & 0.15534 & 0.10321 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
8. (IFGL,SMH) & $1.64\times 10^{-3}$ & 0.01982 & 0.08246 & $1886.02\% \pm 272.20\%$ & 1482.49\% & 1.25 \\
9. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 0.02879 & 0.13849 & $875.54\% \pm 179.98\%$ & 455.31\% & 1.76 \\
10. (IGF,PPH) & $2.29\times 10^{-3}$ & 0.03007 & 0.03391 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
11. (PFF,MBB) & $2.58\times 10^{-3}$ & 0.15600 & 0.09352 & $6.08\% \pm 1.92\%$ & 6.35\% & 1.00 \\
12. (IGF,PKW) & $2.68\times 10^{-3}$ & 0.01972 & 0.05615 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
13. (IFGL,PHO) & $2.70\times 10^{-3}$ & 0.03779 & 0.06508 & $346.16\% \pm 56.61\%$ & 262.45\% & 1.23 \\
14. (IFGL,QQQ) & $2.76\times 10^{-3}$ & 0.01036 & 0.03726 & $-100.00\% \pm 0.00\%$ & 3566.11\% & 0.00 \\
15. (PFF,BND) & $2.78\times 10^{-3}$ & 0.14267 & 0.09753 & $1.79\% \pm 0.67\%$ & 2.88\% & 0.99 \\
16. (PFF,SHY) & $3.04\times 10^{-3}$ & 0.09618 & 0.10584 & $-98.42\% \pm 17.96\%$ & -100.00\% & -1 \\
17. (IFGL,ONEQ) & $3.49\times 10^{-3}$ & 0.01211 & 0.02849 & $358.36\% \pm 62.30\%$ & 389.24\% & 0.94 \\
18. (IGF,FTCS) & $3.83\times 10^{-3}$ & 0.02004 & 0.10683 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
19. (SHY,BND) & $4.05\times 10^{-3}$ & 0.04686 & 0.07972 & $-74.29\% \pm 64.79\%$ & -100.00\% & -1 \\
20. (IGF,PEY) & $4.10\times 10^{-3}$ & 0.05286 & 0.10100 & $-72.37\% \pm 26.19\%$ & -58.46\% & 0.67 \\
21. (IFGL,PKW) & $4.77\times 10^{-3}$ & 0.02679 & 0.01705 & $790.41\% \pm 100.76\%$ & 1039.39\% & 0.78 \\
22. (IFGL,FTCS) & $5.00\times 10^{-3}$ & 0.03208 & 0.03231 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
23. (IGF,PDP) & $5.27\times 10^{-3}$ & 0.02645 & 0.08855 & $2.82\% \pm 157.08\%$ & -100.00\% & -1 \\
24. (IGSB,SHV) & $5.29\times 10^{-3}$ & 0.10288 & 0.02525 & $2.66\% \pm 2.08\%$ & 1.24\% & 1.01 \\
25. (IFGL,USIG) & $5.35\times 10^{-3}$ & 0.14618 & 0.09754 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
26. (IFGL,PDP) & $6.21\times 10^{-3}$ & 0.03299 & 0.02862 & $832.57\% \pm 81.76\%$ & 901.14\% & 0.93 \\
27. (IGF,BBH) & $6.33\times 10^{-3}$ & 0.11199 & 0.05304 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
28. (IFGL,SHY) & $6.79\times 10^{-3}$ & 0.07119 & 0.08850 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
29. (IFGL,PRFZ) & $7.87\times 10^{-3}$ & 0.06617 & 0.04497 & $173.65\% \pm 20.22\%$ & 161.27\% & 1.05 \\
30. (IEF,USIG) & $7.90\times 10^{-3}$ & 0.03858 & 0.05688 & $18.61\% \pm 3.53\%$ & 21.30\% & 0.98 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

"""

transformer_results = latex_to_results(transformer_str)
kalman_results = latex_to_results(kalman_str)
time_moe_results = latex_to_results(time_moe_str)
print("--- Transformer Results: ---")
print_averages(transformer_results)
print("--- Kalman Results: ---")
print_averages(kalman_results)
print("--- Time MOE Results: ---")
print_averages(time_moe_results)

--- Transformer Results: ---
Average val_mse: 1.13770
Average test_mse: 9.40288
Average yoy mean: -93.28500
Average yoy stddev: 0.00000
Average gt_yoy: 211.86000
Average returns_score: 0.18000
--- Kalman Results: ---
Average val_mse: 3.35222
Average test_mse: 3.28207
Average yoy mean: 47.31500
Average yoy stddev: 5.42000
Average gt_yoy: 325.84500
Average returns_score: 0.35000
--- Time MOE Results: ---
Average val_mse: 0.06463
Average test_mse: 0.08129
Average yoy mean: 1.98500
Average yoy stddev: 14.86000
Average gt_yoy: 8.98000
Average returns_score: 0.93500


In [43]:
transformer_alt_str = r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
1. (PFF,EMB) & $1.38\times 10^{-4}$ & 0.11437418 & 0.1663966 & $2.12\% \pm 0.17\%$ & 3.31\% & 0.99 \\
2. (IFGL,MBB) & $4.45\times 10^{-4}$ & 2.1898184 & 1.5658019 & $-91.82\% \pm 51.09\%$ & -100.00\% & -1 \\
3. (IFGL,EMB) & $7.27\times 10^{-4}$ & 0.098761424 & 0.17840905 & $6.49\% \pm 0.70\%$ & 11.61\% & 0.95 \\
4. (IGIB,IEI) & $7.97\times 10^{-4}$ & 0.4119866 & 0.21332882 & $1.26\% \pm 0.08\%$ & 1.38\% & 1.00 \\
5. (IGF,DVY) & $8.40\times 10^{-4}$ & 0.19719256 & 0.31040514 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
6. (USIG,IEI) & $1.09\times 10^{-3}$ & 0.65612984 & 1.9924093 & $-39.57\% \pm 4.59\%$ & -63.21\% & 1.64 \\
7. (IFGL,BND) & $1.20\times 10^{-3}$ & 0.5551524 & 0.6196665 & $-49.14\% \pm 66.64\%$ & -100.00\% & -1 \\
8. (IFGL,SMH) & $1.64\times 10^{-3}$ & 4.461942 & 37.142952 & $164.91\% \pm 7.27\%$ & 1482.49\% & 0.17 \\
9. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 2.8060372 & 33.293037 & $537.63\% \pm 4.76\%$ & 455.31\% & 1.15 \\
10. (IGF,PPH) & $2.29\times 10^{-3}$ & 0.16458306 & 1.1937904 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
11. (PFF,MBB) & $2.58\times 10^{-3}$ & 0.9847346 & 0.6745779 & $2.68\% \pm 0.18\%$ & 6.35\% & 0.97 \\
12. (IGF,PKW) & $2.68\times 10^{-3}$ & 0.0561333 & 5.493732 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
13. (IFGL,PHO) & $2.70\times 10^{-3}$ & 4.8662477 & 35.133392 & $99.30\% \pm 64.83\%$ & 262.45\% & 0.55 \\
14. (IFGL,QQQ) & $2.76\times 10^{-3}$ & 1.5395466 & 18.7831 & $-100.00\% \pm 0.00\%$ & 3566.11\% & 0.00 \\
15. (PFF,BND) & $2.78\times 10^{-3}$ & 0.6438916 & 0.6034082 & $3.93\% \pm 0.24\%$ & 2.88\% & 1.01 \\
16. (PFF,SHY) & $3.04\times 10^{-3}$ & 0.7846692 & 1.7724031 & $-97.73\% \pm 22.67\%$ & -100.00\% & -1 \\
17. (IFGL,ONEQ) & $3.49\times 10^{-3}$ & 2.4131808 & 16.528515 & $129.99\% \pm 2.16\%$ & 389.24\% & 0.47 \\
18. (IGF,FTCS) & $3.83\times 10^{-3}$ & 0.03716507 & 0.69742423 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
19. (SHY,BND) & $4.05\times 10^{-3}$ & 27.956583 & 82.5523 & $0.00\% \pm 0.00\%$ & -100.00\% & -1 \\
20. (IGF,PEY) & $4.10\times 10^{-3}$ & 0.096774004 & 0.11862296 & $-42.25\% \pm 5.86\%$ & -58.46\% & 1.39 \\
21. (IFGL,PKW) & $4.77\times 10^{-3}$ & 3.6971574 & 12.028064 & $-100.00\% \pm 0.00\%$ & 1039.39\% & 0.00 \\
22. (IFGL,FTCS) & $5.00\times 10^{-3}$ & 0.17236567 & 5.9735775 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
23. (IGF,PDP) & $5.27\times 10^{-3}$ & 0.049523998 & 0.4087525 & $538.32\% \pm 6.46\%$ & -100.00\% & -1 \\
24. (IGSB,SHV) & $5.29\times 10^{-3}$ & 0.54342794 & 0.28004116 & $1.75\% \pm 0.34\%$ & 1.24\% & 1.01 \\
25. (IFGL,USIG) & $5.35\times 10^{-3}$ & 0.8215216 & 1.0788443 & $-84.03\% \pm 59.79\%$ & -100.00\% & -1 \\
26. (IFGL,PDP) & $6.21\times 10^{-3}$ & 0.5482751 & 7.300582 & $-100.00\% \pm 0.00\%$ & 901.14\% & 0.00 \\
27. (IGF,BBH) & $6.33\times 10^{-3}$ & 0.5962076 & 0.13683212 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
28. (IFGL,SHY) & $6.79\times 10^{-3}$ & 0.8021414 & 1.2859507 & $-100.00\% \pm 0.00\%$ & -100.00\% & -1 \\
29. (IFGL,PRFZ) & $7.87\times 10^{-3}$ & 5.857771 & 12.645331 & $84.30\% \pm 2.49\%$ & 161.27\% & 0.71 \\
30. (IEF,USIG) & $7.90\times 10^{-3}$ & 0.06186083 & 0.56553805 & $15.22\% \pm 1.08\%$ & 21.30\% & 0.95 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
"""

transformer_alt_results = latex_to_results(transformer_alt_str)
print("--- Transformer Alt Results: ---")
print_averages(transformer_alt_results)

--- Transformer Alt Results: ---
Average val_mse: 1.26214
Average test_mse: 9.66432
Average yoy mean: 4.58500
Average yoy stddev: 0.44000
Average gt_yoy: 211.86000
Average returns_score: 0.63000


In [50]:
time_moe_results_clean = [r for r in time_moe_results if r[-1] != -1 and r[-1] <= 1.0]  
lowest_test_mse = min(r[3] for r in time_moe_results)
lowest_test_mse_index = next(i for i, r in enumerate(time_moe_results) if r[3] == lowest_test_mse)
print(f"Lowest test MSE in time moe Results: {lowest_test_mse} at index {lowest_test_mse_index}")
print(time_moe_results[lowest_test_mse_index])

transformer_alt_results_clean = [r for r in transformer_alt_results if r[-1] != -1 and r[-1] <= 1.0]
lowest_test_mse_alt = min(r[3] for r in transformer_alt_results_clean)
lowest_test_mse_index_alt = next(i for i, r in enumerate(transformer_alt_results_clean) if r[3] == lowest_test_mse_alt)
print(f"Lowest test MSE in transformer alt Results: {lowest_test_mse_alt} at index {lowest_test_mse_index_alt}")
print(transformer_alt_results_clean[lowest_test_mse_index_alt])


Lowest test MSE in time moe Results: 0.01705 at index 20
('21. (IFGL,PKW)', 0.00477, 0.02679, 0.01705, '790.41% +- 100.76%', '1039.39%', 0.78)
Lowest test MSE in transformer alt Results: 0.1663966 at index 0
('1. (PFF,EMB)', 0.000138, 0.11437418, 0.1663966, '2.12% +- 0.17%', '3.31%', 0.99)


In [49]:
transformer_alt_results

[('1. (PFF,EMB)',
  0.000138,
  0.11437418,
  0.1663966,
  '2.12% +- 0.17%',
  '3.31%',
  0.99),
 ('2. (IFGL,MBB)',
  0.00044500000000000003,
  2.1898184,
  1.5658019,
  '-91.82% +- 51.09%',
  '-100.00%',
  -1.0),
 ('3. (IFGL,EMB)',
  0.000727,
  0.098761424,
  0.17840905,
  '6.49% +- 0.70%',
  '11.61%',
  0.95),
 ('4. (IGIB,IEI)',
  0.000797,
  0.4119866,
  0.21332882,
  '1.26% +- 0.08%',
  '1.38%',
  1.0),
 ('5. (IGF,DVY)',
  0.00084,
  0.19719256,
  0.31040514,
  '-100.00% +- 0.00%',
  '-100.00%',
  -1.0),
 ('6. (USIG,IEI)',
  0.00109,
  0.65612984,
  1.9924093,
  '-39.57% +- 4.59%',
  '-63.21%',
  1.64),
 ('7. (IFGL,BND)',
  0.0012,
  0.5551524,
  0.6196665,
  '-49.14% +- 66.64%',
  '-100.00%',
  -1.0),
 ('8. (IFGL,SMH)',
  0.00164,
  4.461942,
  37.142952,
  '164.91% +- 7.27%',
  '1482.49%',
  0.17),
 ('9. (IFGL,SOXX)',
  0.00182,
  2.8060372,
  33.293037,
  '537.63% +- 4.76%',
  '455.31%',
  1.15),
 ('10. (IGF,PPH)',
  0.00229,
  0.16458306,
  1.1937904,
  '-100.00% +- 0.00%',
  

In [27]:
time_moe_results

[('1. (PFF,EMB)',
  0.000138,
  0.10627,
  0.48265,
  '-34.10% +- 48.50%',
  '3.31%',
  0.64),
 ('2. (IFGL,MBB)',
  0.00044500000000000003,
  0.19378,
  3.48831,
  '-100.00% +- 0.00%',
  '-100.00%',
  -1.0),
 ('3. (IFGL,EMB)',
  0.000727,
  0.09752,
  1.61787,
  '-90.39% +- 56.62%',
  '11.61%',
  0.09),
 ('4. (IGIB,IEI)', 0.000797, 0.07641, 0.25635, '1.05% +- 0.20%', '1.38%', 1.0),
 ('5. (IGF,DVY)',
  0.00084,
  0.05064,
  23.59387,
  '-100.00% +- 0.00%',
  '-100.00%',
  -1.0),
 ('6. (USIG,IEI)',
  0.00109,
  0.09513,
  1.17858,
  '-27.96% +- 7.58%',
  '-63.21%',
  1.96),
 ('7. (IFGL,BND)',
  0.0012,
  0.15534,
  4.37671,
  '-100.00% +- 0.00%',
  '-100.00%',
  -1.0),
 ('8. (IFGL,SMH)',
  0.00164,
  0.01982,
  40.98477,
  '-100.00% +- 0.00%',
  '1482.49%',
  0.0),
 ('9. (IFGL,SOXX)',
  0.00182,
  0.02879,
  66.52879,
  '-100.00% +- 0.00%',
  '455.31%',
  0.0),
 ('10. (IGF,PPH)',
  0.00229,
  0.03007,
  11.02186,
  '-100.00% +- 0.00%',
  '-100.00%',
  -1.0),
 ('11. (PFF,MBB)',
  0.002580